In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
import scipy.stats as stats
import sys

sys.path.append("../")
import vuong_tests6

In [2]:
class OLS_loglike(GenericLikelihoodModel):
    
    def __init__(self, *args,ols=False, **kwargs):
        super(OLS_loglike,self).__init__(*args,**kwargs)
        self.ols = ols

    def loglikeobs(self, params):
        y = self.endog
        x = self.exog
        mu_y = np.matmul(x,params)  
        resid = y - mu_y
        sigma = np.sqrt(np.sum(resid**2)/resid.shape[0])
        pr_y = stats.norm.logpdf( resid, loc=0,scale=sigma )
        return pr_y


def setup_shi(yn,xn,return_model=False,num_params=4):
    x1n,x2n = xn[:,0],xn[:,1:num_params+1]
    
    # model 1 grad, etc.
    model1 = sm.OLS(yn,sm.add_constant(x1n))
    model1_fit = model1.fit(disp=False)
    params1 = (model1_fit.params)
    
    model1_deriv = OLS_loglike(yn,sm.add_constant(x1n))
    ll1 = model1_deriv.loglikeobs(model1_fit.params)
    grad1 =  model1_deriv.score_obs(model1_fit.params)    
    hess1 = model1_deriv.hessian(model1_fit.params)
    
    #model 2 grad, etc.
    model2 = sm.OLS(yn,sm.add_constant(x2n))
    model2_fit = model2.fit(disp=False)
    params2 = (model2_fit.params)
    
    model2_deriv = OLS_loglike(yn,sm.add_constant(x2n))
    ll2 = model2_deriv.loglikeobs(model2_fit.params)
    grad2 =  model2_deriv.score_obs(model2_fit.params)    
    hess2 = model2_deriv.hessian(model2_fit.params)
    
    if return_model:
        return ll1,grad1,hess1,params1,model1,ll2,grad2,hess2,params2,model2
    return ll1,grad1,hess1,params1,ll2,grad2,hess2,params2


In [3]:

def gen_data(nobs=1000, a=0.25, num_params=4):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a*x[:,0] + a/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn,return_model=False,num_params=15)
print(grad1.shape,hess1.shape)

(1000, 2) (2, 2)


# a  = .25, k= 9

In [4]:
a = 0.25
num_params=9
num_sims = 100

In [5]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.90 & 0.90 & 0.90 & 0.97 & 0.97 & 0.97   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.01 & 0.01 & 0.01   \\
Model 2 & 0.10 & 0.10 & 0.10 & 0.02 & 0.02 & 0.02   \\
\hline
\end{tabular}
(array([0.9, 0. , 0.1]), array([0.9, 0. , 0.1]), array([0.9, 0. , 0.1]), array([0.97, 0.01, 0.02]), array([0.97, 0.01, 0.02]), array([0.97, 0.01, 0.02]), -4.403234650675367, 5.7900929888320505, 6.045710516943617)


In [6]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.92 & 0.92 & 0.92 & 0.95 & 0.95 & 0.95   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.02 & 0.02 & 0.02   \\
Model 2 & 0.08 & 0.08 & 0.08 & 0.03 & 0.03 & 0.03   \\
\hline
\end{tabular}
(array([0.92, 0.  , 0.08]), array([0.92, 0.  , 0.08]), array([0.92, 0.  , 0.08]), array([0.95, 0.02, 0.03]), array([0.95, 0.02, 0.03]), array([0.95, 0.02, 0.03]), -3.8987929446480085, 8.566345531439161, 8.171437988783271)


In [7]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=100, a=a,  num_params=num_params)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.93 & 0.97 & 0.91 & 0.98 & 0.98 & 0.99   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.02 & 0.02 & 0.01   \\
Model 2 & 0.07 & 0.03 & 0.09 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.93, 0.  , 0.07]), array([0.97, 0.  , 0.03]), array([0.91, 0.  , 0.09]), array([0.98, 0.02, 0.  ]), array([0.98, 0.02, 0.  ]), array([0.99, 0.01, 0.  ]), -3.908269452226756, 3.819862492664278, 4.368693425182368)


# a  = .25, k= 4, something not right?

In [8]:
a = 0.25
num_params=4
num_sims = 100

In [9]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.97 & 0.97 & 0.94 & 0.97 & 0.98 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.01 & 0.00 & 0.00   \\
Model 2 & 0.03 & 0.03 & 0.06 & 0.02 & 0.02 & 0.00   \\
\hline
\end{tabular}
(array([0.97, 0.  , 0.03]), array([0.97, 0.  , 0.03]), array([0.94, 0.  , 0.06]), array([0.97, 0.01, 0.02]), array([0.98, 0.  , 0.02]), array([1., 0., 0.]), -1.387177459513802, 5.058991391432905, 5.601190541278347)


In [10]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.97 & 0.97 & 0.96 & 0.97 & 0.97 & 0.97   \\
Model 1 & 0.02 & 0.02 & 0.02 & 0.02 & 0.02 & 0.02   \\
Model 2 & 0.01 & 0.01 & 0.02 & 0.01 & 0.01 & 0.01   \\
\hline
\end{tabular}
(array([0.97, 0.02, 0.01]), array([0.97, 0.02, 0.01]), array([0.96, 0.02, 0.02]), array([0.97, 0.02, 0.01]), array([0.97, 0.02, 0.01]), array([0.97, 0.02, 0.01]), -0.3132471432999189, 7.503649787839507, 7.726989037371383)


# a  = .25, K = 19

In [11]:
a = 0.25
num_params=19
num_sims = 100

In [12]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.69 & 0.83 & 0.69 & 0.97 & 0.97 & 0.98   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.02 & 0.02 & 0.01   \\
Model 2 & 0.31 & 0.17 & 0.31 & 0.01 & 0.01 & 0.01   \\
\hline
\end{tabular}
(array([0.69, 0.  , 0.31]), array([0.83, 0.  , 0.17]), array([0.69, 0.  , 0.31]), array([0.97, 0.02, 0.01]), array([0.97, 0.02, 0.01]), array([0.98, 0.01, 0.01]), -9.52355003465035, 5.736313076482998, 6.585628859296977)


In [13]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.80 & 0.80 & 0.78 & 0.97 & 0.98 & 0.98   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.01 & 0.00 & 0.00   \\
Model 2 & 0.20 & 0.20 & 0.22 & 0.02 & 0.02 & 0.02   \\
\hline
\end{tabular}
(array([0.8, 0. , 0.2]), array([0.8, 0. , 0.2]), array([0.78, 0.  , 0.22]), array([0.97, 0.01, 0.02]), array([0.98, 0.  , 0.02]), array([0.98, 0.  , 0.02]), -9.700693127146216, 8.226062956766746, 8.653749937879446)


# evidence of power

In [14]:
a1,a2 = np.sqrt(1.09-1), 0.00
num_params= 9
num_sims = 100

def gen_data2(nobs=1000, a1=np.sqrt(1.09-1), a2=0.00 , num_params=19):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a1*x[:,0] + a2/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

In [15]:
a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.87 & 0.88 & 0.86 & 0.53 & 0.53 & 0.61   \\
Model 1 & 0.13 & 0.12 & 0.14 & 0.47 & 0.47 & 0.39   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.87, 0.13, 0.  ]), array([0.88, 0.12, 0.  ]), array([0.86, 0.14, 0.  ]), array([0.53, 0.47, 0.  ]), array([0.53, 0.47, 0.  ]), array([0.61, 0.39, 0.  ]), 5.8299751166445, 4.608457397021836, 5.263192215816153)


In [16]:
a1,a2 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=500, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.83 & 0.83 & 0.81 & 0.37 & 0.39 & 0.48   \\
Model 1 & 0.17 & 0.17 & 0.19 & 0.63 & 0.61 & 0.52   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.83, 0.17, 0.  ]), array([0.83, 0.17, 0.  ]), array([0.81, 0.19, 0.  ]), array([0.37, 0.63, 0.  ]), array([0.39, 0.61, 0.  ]), array([0.48, 0.52, 0.  ]), 7.236665449414542, 4.771886023101383, 5.56093930427876)


In [17]:
a1,a2 = np.sqrt(1.09**2.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=100, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.81 & 0.89 & 0.81 & 0.48 & 0.49 & 0.56   \\
Model 1 & 0.19 & 0.11 & 0.19 & 0.52 & 0.51 & 0.44   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.81, 0.19, 0.  ]), array([0.89, 0.11, 0.  ]), array([0.81, 0.19, 0.  ]), array([0.48, 0.52, 0.  ]), array([0.49, 0.51, 0.  ]), array([0.56, 0.44, 0.  ]), 5.764350901158788, 4.793504893615165, 4.917438468229537)


In [18]:
a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=4)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.55 & 0.55 & 0.54 & 0.35 & 0.37 & 0.54   \\
Model 1 & 0.45 & 0.45 & 0.46 & 0.65 & 0.63 & 0.46   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.55, 0.45, 0.  ]), array([0.55, 0.45, 0.  ]), array([0.54, 0.46, 0.  ]), array([0.35, 0.65, 0.  ]), array([0.37, 0.63, 0.  ]), array([0.54, 0.46, 0.  ]), 9.901782207779071, 5.039358656795219, 5.06855025966164)


In [19]:
a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=19)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.79 & 0.79 & 0.77 & 0.43 & 0.44 & 0.50   \\
Model 1 & 0.21 & 0.21 & 0.23 & 0.57 & 0.56 & 0.50   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.79, 0.21, 0.  ]), array([0.79, 0.21, 0.  ]), array([0.77, 0.23, 0.  ]), array([0.43, 0.57, 0.  ]), array([0.44, 0.56, 0.  ]), array([0.5, 0.5, 0. ]), 7.145263153436763, 4.5730439546109904, 5.40326249124368)


# evidence of power 2

In [20]:
a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.13 & 0.22 & 0.12 & 0.56 & 0.56 & 0.63   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.87 & 0.78 & 0.88 & 0.44 & 0.44 & 0.37   \\
\hline
\end{tabular}
(array([0.13, 0.  , 0.87]), array([0.22, 0.  , 0.78]), array([0.12, 0.  , 0.88]), array([0.56, 0.  , 0.44]), array([0.56, 0.  , 0.44]), array([0.63, 0.  , 0.37]), -14.369646319078289, 4.897895557797232, 5.321142029972063)


In [21]:
a2,a1 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=500, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.06 & 0.14 & 0.08 & 0.50 & 0.52 & 0.61   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.94 & 0.86 & 0.92 & 0.50 & 0.48 & 0.39   \\
\hline
\end{tabular}
(array([0.06, 0.  , 0.94]), array([0.14, 0.  , 0.86]), array([0.08, 0.  , 0.92]), array([0.5, 0. , 0.5]), array([0.52, 0.  , 0.48]), array([0.61, 0.  , 0.39]), -14.878521734257804, 5.141261820665246, 5.448397087647558)


In [22]:
a2,a1 =np.sqrt(1.09**2.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=100, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.08 & 0.37 & 0.09 & 0.40 & 0.40 & 0.46   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.92 & 0.63 & 0.91 & 0.60 & 0.60 & 0.54   \\
\hline
\end{tabular}
(array([0.08, 0.  , 0.92]), array([0.37, 0.  , 0.63]), array([0.09, 0.  , 0.91]), array([0.4, 0. , 0.6]), array([0.4, 0. , 0.6]), array([0.46, 0.  , 0.54]), -15.888315242752437, 5.8222780620444725, 5.213169858761685)


In [23]:
a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=4)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.19 & 0.19 & 0.19 & 0.41 & 0.41 & 0.55   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.81 & 0.81 & 0.81 & 0.59 & 0.59 & 0.45   \\
\hline
\end{tabular}
(array([0.19, 0.  , 0.81]), array([0.19, 0.  , 0.81]), array([0.19, 0.  , 0.81]), array([0.41, 0.  , 0.59]), array([0.41, 0.  , 0.59]), array([0.55, 0.  , 0.45]), -12.355795335401362, 4.940099466523286, 4.893477866486179)


In [25]:
a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=4)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.45 & 0.72 & 0.43 & 0.85 & 0.86 & 0.88   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.55 & 0.28 & 0.57 & 0.15 & 0.14 & 0.12   \\
\hline
\end{tabular}
(array([0.45, 0.  , 0.55]), array([0.72, 0.  , 0.28]), array([0.43, 0.  , 0.57]), array([0.85, 0.  , 0.15]), array([0.86, 0.  , 0.14]), array([0.88, 0.  , 0.12]), -9.272729628765545, 4.4773283933250685, 4.322096832267187)
